In [ ]:
#Read JSON file
import pandas as pd
import json
from pandas.io.json import json_normalize

with open(r"C:\Users\User\Documents\Sample.json") as data_file:
    data = json.load(data_file)

#Create Dataframe from JSON file; this should display the first level nested keys and values
df = pd.io.json.json_normalize(data)
df = json_normalize(data)

#Flatten nested fields
def flatten_json(nested_json, exclude=['']):
    out = {}
    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type (x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) +'_')
                i += 1
        else:
            out[name[:-1]] = x    
    flatten(nested_json)
    return out

#Isolate the 1-level nested keys from dataframe "df"
df1 = df[["Display Name", "Project Name", "Project Status"]]

#Isolate the 1-level nested key "Referenced Endpoints"
df_endpoint = pd.DataFrame([flatten_json(x) for x in data["Referenced Endpoints"]])

#Isolate the 1-level nested key "Referenced Reports"
df_ref = pd.DataFrame([flatten_json(x) for x in data["Referenced Reports"]])

#Isolate the 2-level nested key "Reports"
df_reports = pd.DataFrame([flatten_json(x) for x in data["Reports"]])

#Flatten instead of normalize the JSON string "data"
flat = flatten_json(data)
df2 = json_normalize(flat)

#Isolate the 2-level nested key "Access Control_Groups_0_Description" and "Access Control_Groups_0_Group Name"
df_access = df2[["Access Control_Groups_0_Description", "Access Control_Groups_0_Group Name"]]

#Isolate and transpose the 2-level nested key "Access Control_Groups_0_Membership"
df_access2 = df2.filter(regex="Access Control_Groups_0_Membership", axis=1).head()
df_access2_transposed = df_access2.T
#Assign header as "Access Control_Groups_0_Membership"
df_access2_transposed.columns = ["Access Control_Groups_0_Membership"]
df_access2_transposed.reset_index(drop=True, inplace=True)

#Dataframe to Excel
from pandas import ExcelWriter
writer = ExcelWriter(r"C:\Users\User\Documents\Sample.xlsx")
df_access.to_excel(writer, "Access Control", index=False)
df_access2_transposed.to_excel(writer, "Access Control Membership", index=False)
df1.to_excel(writer, "Other Columns", index=False)
df_endpoint.to_excel(writer, "Referenced Endpoints", index=False)
df_ref.to_excel(writer, "Referenced Reports", index=False)
df_reports.to_excel(writer, "Reports", index=False)
writer.save()